In [11]:
# Ray Tune
from functools import partial
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from torch.utils.data import random_split
import os

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision.transforms import *
from torch.utils.data import DataLoader

import numpy as np

In [2]:
# BATCH_SIZE = 64
# EPOCHS = 3
# LEARNING_RATE = 0.001
BATCH_INTERVAL = 100
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000
log_interval = 100

In [3]:
transform = transforms.Compose([transforms.Resize([28, 28]),
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

def get_dataloader(train_set, BATCH_SIZE):
    return torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

# Load data

In [19]:
def load_data(data_dir="~/Anders/MasterBSB/2210_DeepLearning/Ass3/CNN/data/mnist-varres/"):
    transform = transforms.Compose([transforms.Resize([28, 28]),
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])
    trainset = torchvision.datasets.ImageFolder(root = "~/Anders/MasterBSB/2210_DeepLearning/Ass3/CNN/data/mnist-varres/train/",
                                               transform = transform)
    testset = torchvision.datasets.ImageFolder(root = "~/Anders/MasterBSB/2210_DeepLearning/Ass3/CNN/data/mnist-varres/test/",
                                               transform=transform)
    return trainset, testset

In [ ]:
# def get_distribution(train_size, val_size, total_size):
#     sizes = [math.floor(train_size/(train_size+val_size)*total_size),
#      math.ceil((1-train_size/(train_size+val_size))*total_size)]
#     return sizes

In [ ]:
# dataset = torchvision.datasets.ImageFolder(root = "../data/mnist-varres/train/")

# train_set, validation_set = torch.utils.data.random_split(dataset, [TRAIN_SIZE, VALIDATION_SIZE])

# train_loader = torch.utils.data.DataLoader(train_set,
#                                            batch_size = BATCH_SIZE,
#                                            shuffle = True,
#                                            num_workers = 2)

# validation_loader = torch.utils.data.DataLoader(validation_set,
#                                                 batch_size = BATCH_SIZE,
#                                                 shuffle = True,
#                                                 num_workers = 2)

In [ ]:
# test_set = torchvision.datasets.ImageFolder(root = "../data/mnist-varres/test/",
#                                           transform=transform)

# test_loader = torch.utils.data.DataLoader(test_set,
#                                           batch_size = BATCH_SIZE,
#                                           shuffle = True,
#                                           num_workers = 2)

# Network architecture

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1) # size: batch, 1, 28, 28
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1) # (batch, 16, 14, 14)
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1) # (batch, 32, 7, 7)
        self.fc1 = nn.Linear(64*3*3, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # (batch, 16, 28, 28) # TODO klopt het dat relu en maxpool zijn omgedraaid?
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # (batch, 32, 14, 14)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2) # (batch, 64, 7, 7)
        x = torch.flatten(x, 1) # (batch, 64, 3, 3)
        x = self.fc1(x)
        return F.log_softmax(x, dim = 1)

In [29]:
# config: the hyperparameters we would like to train with
# checkpoint_dir: used to restore checkpoints
# data_dir: the directory where we load and store the data, so multiple runs can share the same data source
def train_mnist(config, checkpoint_dir=None, data_dir=None):
    net = Net()
#     net = Net(config["l1"], config["l2"]) # previously

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr = config["lr"], weight_decay = config["weight_decay"])

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    # Load data.
    trainset, testset = load_data()
    traindata, valdata = torch.utils.data.random_split(trainset, [TRAIN_SIZE, VALIDATION_SIZE])

    train_loader = torch.utils.data.DataLoader(traindata,
                                               batch_size = int(config["batch_size"]),
                                               shuffle = True,
                                               num_workers = 2)
    validation_loader = torch.utils.data.DataLoader(valdata,
                                                batch_size = int(config["batch_size"]),
                                                shuffle = True,
                                                num_workers = 2)
        
    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0

        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(validation_loader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [30]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [31]:
# Full function:
def main(num_samples = 10, max_num_epochs = 10, gpus_per_trial = 0):
    data_dir = os.path.abspath("~/Anders/MasterBSB/2210_DeepLearning/Ass3/CNN/data/mnist-varres/")
    
    load_data() # Hier wordt alleen testdata goed geladen, traindata is dummy
    
    # Define hyperparameter search space.
    config = {
    #     "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    #     "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16, 32, 64]),
        "weight_decay": tune.choice([0, 0.01])
    }
    # In this example, the l1 and l2 parameters should be powers of 2 between 4 and 256, so either
    # 4, 8, 16, 32, 64, 128, or 256.
    # The lr (learning rate) should be uniformly sampled between 0.0001 and 0.1
    
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_mnist, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    
    best_trained_model = Net() # removed best_trial.config["l1"], best_trial.config["l2"]
    device = "cpu"
    best_trained_model.to(device)
    
    print("Path to best model: {}".format(os.path.join(best_checkpoint_dir, "checkpoint")))
    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)
    
    test_acc = test_accuracy(best_trained_model, device) # dit is op test set, voor nu eruit
    print("Best trial test set accuracy: {}".format(test_acc))

In [32]:
main()

== Status ==
Current time: 2022-12-12 12:01:52 (running for 00:00:00.15)
Memory usage on this node: 12.3/16.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/8 CPUs, 0/0 GPUs, 0.0/6.43 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/jannekehulsen/ray_results/train_mnist_2022-12-12_12-01-52
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+----------------+
| Trial name              | status   | loc             |   batch_size |          lr |   weight_decay |
|-------------------------+----------+-----------------+--------------+-------------+----------------|
| train_mnist_621fc_00000 | RUNNING  | 127.0.0.1:29476 |           64 | 0.0595751   |           0.01 |
| train_mnist_621fc_00001 | PENDING  |                 |            8 | 0.00652755  |           0    |
| train_mnist_621fc_00002 | PENDING  |    

== Status ==
Current time: 2022-12-12 12:02:18 (running for 00:00:26.55)
Memory usage on this node: 13.1/16.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/6.43 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/jannekehulsen/ray_results/train_mnist_2022-12-12_12-01-52
Number of trials: 10/10 (6 PENDING, 4 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+----------------+
| Trial name              | status   | loc             |   batch_size |          lr |   weight_decay |
|-------------------------+----------+-----------------+--------------+-------------+----------------|
| train_mnist_621fc_00000 | RUNNING  | 127.0.0.1:29476 |           64 | 0.0595751   |           0.01 |
| train_mnist_621fc_00001 | RUNNING  | 127.0.0.1:29483 |            8 | 0.00652755  |           0    |
| train_mnist_621fc_00002 | RUNNING  | 127

== Status ==
Current time: 2022-12-12 12:02:43 (running for 00:00:51.65)
Memory usage on this node: 12.9/16.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/6.43 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/jannekehulsen/ray_results/train_mnist_2022-12-12_12-01-52
Number of trials: 10/10 (6 PENDING, 4 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+----------------+
| Trial name              | status   | loc             |   batch_size |          lr |   weight_decay |
|-------------------------+----------+-----------------+--------------+-------------+----------------|
| train_mnist_621fc_00000 | RUNNING  | 127.0.0.1:29476 |           64 | 0.0595751   |           0.01 |
| train_mnist_621fc_00001 | RUNNING  | 127.0.0.1:29483 |            8 | 0.00652755  |           0    |
| train_mnist_621fc_00002 | RUNNING  | 127

Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_mnist_621fc_00000,0.0964,2022-12-12_12-10-21,True,,a4fa52684fbe44deb31ddbe6d45d6af8,feweb60177.feweb.vu.nl,10,2.30808,127.0.0.1,29476,True,507.582,53.6624,507.582,1670843421,0,,10,621fc_00000,0.0016849
train_mnist_621fc_00001,0.8334,2022-12-12_12-11-15,True,,83db23eb9bd941f5829ce4337a68e243,feweb60177.feweb.vu.nl,10,0.581276,127.0.0.1,29483,True,560.664,53.4082,560.664,1670843475,0,,10,621fc_00001,0.003052
train_mnist_621fc_00002,0.9722,2022-12-12_12-10-44,True,,d02f743c2e5147e2a4d94d7940cf366f,feweb60177.feweb.vu.nl,10,0.130059,127.0.0.1,29484,True,529.694,49.8381,529.694,1670843444,0,,10,621fc_00002,0.0030961
train_mnist_621fc_00003,0.1097,2022-12-12_12-03-43,True,,6f9f37a29a344c4ba0f07a75bcafbb45,feweb60177.feweb.vu.nl,2,2.31064,127.0.0.1,29485,True,108.279,52.9799,108.279,1670843023,0,,2,621fc_00003,0.00207686
train_mnist_621fc_00004,0.6989,2022-12-12_12-07-15,True,,6f9f37a29a344c4ba0f07a75bcafbb45,feweb60177.feweb.vu.nl,4,0.914862,127.0.0.1,29485,True,211.688,52.9391,211.688,1670843235,0,,4,621fc_00004,0.00207686
train_mnist_621fc_00005,0.1124,2022-12-12_12-08-31,True,,6f9f37a29a344c4ba0f07a75bcafbb45,feweb60177.feweb.vu.nl,1,2.30238,127.0.0.1,29485,True,76.0924,76.0924,76.0924,1670843311,0,,1,621fc_00005,0.00207686
train_mnist_621fc_00006,0.7144,2022-12-12_12-11-52,True,,6f9f37a29a344c4ba0f07a75bcafbb45,feweb60177.feweb.vu.nl,4,0.856755,127.0.0.1,29485,True,201.026,47.2026,201.026,1670843512,0,,4,621fc_00006,0.00207686
train_mnist_621fc_00007,0.112,2022-12-12_12-11-11,True,,a4fa52684fbe44deb31ddbe6d45d6af8,feweb60177.feweb.vu.nl,1,2.30225,127.0.0.1,29476,True,49.9113,49.9113,49.9113,1670843471,0,,1,621fc_00007,0.0016849
train_mnist_621fc_00008,0.9475,2022-12-12_12-18-32,True,,d02f743c2e5147e2a4d94d7940cf366f,feweb60177.feweb.vu.nl,10,0.206519,127.0.0.1,29484,True,467.725,46.9711,467.725,1670843912,0,,10,621fc_00008,0.0030961
train_mnist_621fc_00009,0.9287,2022-12-12_12-18-48,True,,a4fa52684fbe44deb31ddbe6d45d6af8,feweb60177.feweb.vu.nl,10,0.296136,127.0.0.1,29476,True,456.866,46.6032,456.866,1670843928,0,,10,621fc_00009,0.0016849


== Status ==
Current time: 2022-12-12 12:02:50 (running for 00:00:58.32)
Memory usage on this node: 13.0/16.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -2.3125835231914644
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/6.43 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/jannekehulsen/ray_results/train_mnist_2022-12-12_12-01-52
Number of trials: 10/10 (6 PENDING, 4 RUNNING)
+-------------------------+----------+-----------------+--------------+-------------+----------------+---------+------------+----------------------+
| Trial name              | status   | loc             |   batch_size |          lr |   weight_decay |    loss |   accuracy |   training_iteration |
|-------------------------+----------+-----------------+--------------+-------------+----------------+---------+------------+----------------------|
| train_mnist_621fc_00000 | RUNNING  | 127.0.0.1:29476 |           64 | 0.0595751   |          

...

(func pid=29484) [10,  6000] loss: 0.068
== Status ==
Current time: 2022-12-12 12:18:11 (running for 00:16:19.28)
Memory usage on this node: 12.5/16.0 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 8.000: -0.27092795160543176 | Iter 4.000: -0.443901959973108 | Iter 2.000: -0.6590621637462004 | Iter 1.000: -0.8197678822208363
Resources requested: 4.0/8 CPUs, 0/0 GPUs, 0.0/6.43 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/jannekehulsen/ray_results/train_mnist_2022-12-12_12-01-52
Number of trials: 10/10 (2 RUNNING, 8 TERMINATED)
+-------------------------+------------+-----------------+--------------+-------------+----------------+----------+------------+----------------------+
| Trial name              | status     | loc             |   batch_size |          lr |   weight_decay |     loss |   accuracy |   training_iteration |
|-------------------------+------------+-----------------+--------------+-------------+----------------+----------+------------+---------------------

== Status ==
Current time: 2022-12-12 12:18:31 (running for 00:16:39.36)
Memory usage on this node: 12.5/16.0 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 8.000: -0.27092795160543176 | Iter 4.000: -0.443901959973108 | Iter 2.000: -0.6590621637462004 | Iter 1.000: -0.8197678822208363
Resources requested: 4.0/8 CPUs, 0/0 GPUs, 0.0/6.43 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/jannekehulsen/ray_results/train_mnist_2022-12-12_12-01-52
Number of trials: 10/10 (2 RUNNING, 8 TERMINATED)
+-------------------------+------------+-----------------+--------------+-------------+----------------+----------+------------+----------------------+
| Trial name              | status     | loc             |   batch_size |          lr |   weight_decay |     loss |   accuracy |   training_iteration |
|-------------------------+------------+-----------------+--------------+-------------+----------------+----------+------------+----------------------|
| train_mnist_621fc_00008 | RUNNING   

2022-12-12 12:18:48,263	INFO tune.py:777 -- Total run time: 1015.99 seconds (1015.86 seconds for the tuning loop).


== Status ==
Current time: 2022-12-12 12:18:48 (running for 00:16:55.87)
Memory usage on this node: 10.9/16.0 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -0.27092795160543176 | Iter 4.000: -0.443901959973108 | Iter 2.000: -0.6590621637462004 | Iter 1.000: -0.8197678822208363
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/6.43 GiB heap, 0.0/2.0 GiB objects
Result logdir: /Users/jannekehulsen/ray_results/train_mnist_2022-12-12_12-01-52
Number of trials: 10/10 (10 TERMINATED)
+-------------------------+------------+-----------------+--------------+-------------+----------------+----------+------------+----------------------+
| Trial name              | status     | loc             |   batch_size |          lr |   weight_decay |     loss |   accuracy |   training_iteration |
|-------------------------+------------+-----------------+--------------+-------------+----------------+----------+------------+----------------------|
| train_mnist_621fc_00000 | TERMINATED | 127.0.0.

UnboundLocalError: local variable 'best_checkpoint_dir' referenced before assignment